In [1]:
!pip3 install folium
!pip3 install wget


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import folium
import wget
import pandas as pd
import os

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [5]:
df = pd.read_csv('data_Falcon9.csv')
df

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,1316.0,LEO,CCSFS SLC 40,None - None,1,False,False,False,1.0,0,B0003,-80.577366,28.561857,0
1,2,2010-12-08,Falcon 9,1316.0,LEO,CCSFS SLC 40,None - None,1,False,False,False,1.0,0,B0004,-80.577366,28.561857,0
2,3,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None - None,1,False,False,False,1.0,0,B0005,-80.577366,28.561857,0
3,4,2012-10-08,Falcon 9,800.0,LEO,CCSFS SLC 40,None - None,1,False,False,False,1.0,0,B0006,-80.577366,28.561857,0
4,5,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None - None,1,False,False,False,1.0,0,B0007,-80.577366,28.561857,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,175,2022-09-05,Falcon 9,13440.0,VLEO,CCSFS SLC 40,True - ASDS,7,True,True,True,5.0,6,B1052,-80.577366,28.561857,1
173,176,2022-09-11,Falcon 9,14760.0,VLEO,KSC LC 39A,True - ASDS,14,True,True,True,5.0,13,B1058,-80.603956,28.608058,1
174,177,2022-09-17,Falcon 9,13260.0,VLEO,CCSFS SLC 40,True - ASDS,6,True,True,True,5.0,5,B1067,-80.577366,28.561857,1
175,178,2022-09-24,Falcon 9,13260.0,VLEO,CCSFS SLC 40,True - ASDS,4,True,True,True,5.0,0,B1072,-80.577366,28.561857,1


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
falcon9_df = df[['FlightNumber','Date','Orbit','LaunchSite', 'Latitude', 'Longitude', 'Class']]
launch_sites_df = falcon9_df.groupby(['LaunchSite'], as_index=False).first()
launch_sites_df = launch_sites_df[['LaunchSite', 'Latitude', 'Longitude']]
launch_sites_df

,LaunchSite,Latitude,Longitude
0,CCSFS SLC 40,28.561857,-80.577366
1,KSC LC 39A,28.608058,-80.603956
2,VAFB SLC 4E,34.632093,-120.610829


In [7]:
# Calculate success rate for each launch site
success_rate = falcon9_df.groupby('LaunchSite')['Class'].mean()

# Create a map centered around NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, site in launch_sites_df.iterrows():
    cord = [site['Latitude'],site['Longitude']]
    
    # Calculate success rate for this site
    site_success_rate = success_rate[site['LaunchSite']]
    
    circle = folium.Circle(location = cord, radius=150, color='#d35400', fill=True).add_child( folium.Popup(
            f"Launch Site: {site['LaunchSite']}<br>"
            f"Success Rate: {site_success_rate:.2f}"
        ))
    # code for the marker
    marker = folium.map.Marker(cord,
                               icon=DivIcon(
                                   icon_size=(20,20), 
                                   icon_anchor=(0,0),
                                   html='<div style="font-size: 16; color:#d35400;"><b>%s</b></div>' % site['LaunchSite'],
                               )
                              )
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map



In [8]:
falcon9_df 

,FlightNumber,Date,Orbit,LaunchSite,Latitude,Longitude,Class
0,1,2010-06-04,LEO,CCSFS SLC 40,28.561857,-80.577366,0
1,2,2010-12-08,LEO,CCSFS SLC 40,28.561857,-80.577366,0
2,3,2012-05-22,LEO,CCSFS SLC 40,28.561857,-80.577366,0
3,4,2012-10-08,LEO,CCSFS SLC 40,28.561857,-80.577366,0
4,5,2013-03-01,ISS,CCSFS SLC 40,28.561857,-80.577366,0
...,...,...,...,...,...,...,...
172,175,2022-09-05,VLEO,CCSFS SLC 40,28.561857,-80.577366,1
173,176,2022-09-11,VLEO,KSC LC 39A,28.608058,-80.603956,1
174,177,2022-09-17,VLEO,CCSFS SLC 40,28.561857,-80.577366,1
175,178,2022-09-24,VLEO,CCSFS SLC 40,28.561857,-80.577366,1


In [9]:
marker_cluster = MarkerCluster()

In [10]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
    
falcon9_df['marker_color'] = falcon9_df['Class'].apply(assign_marker_color)
falcon9_df


/var/folders/p0/kl_xr30549z8n56kftyqjnkh0000gn/T/ipykernel_35615/2044850781.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  falcon9_df['marker_color'] = falcon9_df['Class'].apply(assign_marker_color)


,FlightNumber,Date,Orbit,LaunchSite,Latitude,Longitude,Class,marker_color
0,1,2010-06-04,LEO,CCSFS SLC 40,28.561857,-80.577366,0,red
1,2,2010-12-08,LEO,CCSFS SLC 40,28.561857,-80.577366,0,red
2,3,2012-05-22,LEO,CCSFS SLC 40,28.561857,-80.577366,0,red
3,4,2012-10-08,LEO,CCSFS SLC 40,28.561857,-80.577366,0,red
4,5,2013-03-01,ISS,CCSFS SLC 40,28.561857,-80.577366,0,red
...,...,...,...,...,...,...,...,...
172,175,2022-09-05,VLEO,CCSFS SLC 40,28.561857,-80.577366,1,green
173,176,2022-09-11,VLEO,KSC LC 39A,28.608058,-80.603956,1,green
174,177,2022-09-17,VLEO,CCSFS SLC 40,28.561857,-80.577366,1,green
175,178,2022-09-24,VLEO,CCSFS SLC 40,28.561857,-80.577366,1,green


In [11]:
# Using GroupBy & count() on multiple column
df_count = falcon9_df.groupby(['LaunchSite','Class'])['Class'].count()
#print(df_count)
df_count.to_frame()

Class
LaunchSite   Class       
CCSFS SLC 40 0         27
             1         72
KSC LC 39A   0          4
             1         46
VAFB SLC 4E  0          5
             1         23

In [12]:
# Clear existing marker_cluster and create a new one
marker_cluster = MarkerCluster()


for index, record in falcon9_df.iterrows():
   
    # Create a Marker object with its coordinate
    marker = folium.Marker(
        location=[record['Latitude'], record['Longitude']],
         popup=f'''
            <strong>Flight Number:</strong> {record['FlightNumber']}<br>
            <strong>Date:</strong> {record['Date']}<br>
            <strong>Orbit:</strong> {record['Orbit']}<br>
        '''
    )
    # Customize the Marker's icon property to indicate success or failure
    icon = folium.Icon(color='white', icon_color=record['marker_color']).add_to(marker)
     

    # Add the marker to the marker_cluster
    marker.add_to(marker_cluster)
# Add the updated marker_cluster to the map
site_map.add_child(marker_cluster)